#### Data Class

In [14]:
import random

class Sentiment:
    Negative = "Negative"
    Neutral = "Neutral"
    Positive = "Positive"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.Negative
        elif self.score == 3:
            return Sentiment.Neutral
        else: #Score of 4 or 5
            return Sentiment.Positive

#After loading new data, creating a new class

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):#count the no. negative sentiments and positive 
        negative = list(filter(lambda x: x.sentiment == Sentiment.Negative, self.reviews)) # mapping every negative sentiment
        positive = list(filter(lambda x: x.sentiment == Sentiment.Positive, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

### Load Data

In [15]:
import json
import numpy as np
file_name = "C:/Users/Akshat/Desktop/Books_small_10000.json" #loaded data with more samples as to even the +ve and -ve ratio coz the previous data was causing and was biased towards +ve reviews

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
print("Review by user:", reviews[5].text)
print("=========")
print("Rating:", reviews[5].score)
print("=========")
print("Sentiment:", reviews[5].sentiment)
#Also we can do:
        #print(review['reviewText'])
        #print('=========')
        #print("Review Rating:", review['overall'])
        #break

Review by user: I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
Rating: 5.0
Sentiment: Positive


### Prep Data

In [16]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size = 0.33, random_state=42)


train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

In [17]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()

test_x = train_container.get_text()
test_y = train_container.get_sentiment()

print(train_y.count(Sentiment.Positive))
print(train_y.count(Sentiment.Negative))

436
436


### Bag of words vectorization

In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


vectorizer = TfidfVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

#train_x_vectors
#train_y

#print(vectorizer.get_feature_names())

I loved this series! It's so fun to read the books in order because each story introduces the main character of the next one. I loved them. The last was my favorite. At first I judged the books by the, um, very unimpressive book covers and passed on them. I'm so glad I finally decided to give them a try. Lots of fun, so sweet! I really love the concept of one story leading to the next. I wish there was another series like this. Definitely read them in order and try to buy them together in the pack so you have them all together. Then, sit back, read straight through, and relax into the stories. The only flaw I found in most is that they weren't longer.
[[0. 0. 0. ... 0. 0. 0.]]


### Classification

#### Linear SVM

In [29]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

### Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

### Naive Bayes

In [31]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.todense(), train_y)

clf_gnb.predict(test_x_vectors[0].todense())

array(['Positive'], dtype='<U8')

### Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(train_x_vectors[0])

array(['Positive'], dtype='<U8')

### Evaluation

In [54]:
# Mean Accuracy

#print("SVM Score:",clf_svm.score(test_x_vectors, test_y))
#print("Decision Tree Score:",clf_dec.score(test_x_vectors, test_y))
#print("NaiveBayes Score:",clf_gnb.score(test_x_vectors.todense(), test_y))
#print("Logistic Regression Score:",clf_log.score(test_x_vectors, test_y))

print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.todense(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.9885321100917431
1.0
0.9839449541284404
0.9655963302752294


In [55]:

# F1 Score
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])
#f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])
#f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])
#f1_score(test_y, clf_gnb.predict(test_x_vectors.todense()), average=None, labels=[Sentiment.Positive, Sentiment.Neutral, Sentiment.Negative])

C:\Users\Akshat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


array([0.98850575, 0.        , 0.98855835])

In [35]:
test_set = ['very fun', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['Positive', 'Negative', 'Negative'], dtype='<U8')

### Tuninng Model with GridSearch

In [37]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C':(1,4,8,16,32)}

svc = svm.SVC()

clf = GridSearchCV(svc, parameters)
clf.fit(train_x_vectors, train_y)

GridSearchCV(estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [56]:
print(clf.score(test_x_vectors, test_y))

1.0


### Saving the model

In [50]:
import pickle

with open('C:/Users/Akshat/Desktop/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

### Load Model

In [51]:
with open('C:/Users/Akshat/Desktop/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [53]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

I loved this series! It's so fun to read the books in order because each story introduces the main character of the next one. I loved them. The last was my favorite. At first I judged the books by the, um, very unimpressive book covers and passed on them. I'm so glad I finally decided to give them a try. Lots of fun, so sweet! I really love the concept of one story leading to the next. I wish there was another series like this. Definitely read them in order and try to buy them together in the pack so you have them all together. Then, sit back, read straight through, and relax into the stories. The only flaw I found in most is that they weren't longer.


array(['Positive'], dtype='<U8')